In [28]:
import pandas as pd
import re
from functools import reduce

In [5]:
def get_input(test_data=None):
    data = test_data if test_data is not None else open('input.txt').read()
    return [n for n in data.split('\n\n')]

In [6]:
test_data = '''class: 1-3 or 5-7
row: 6-11 or 33-44
seat: 13-40 or 45-50

your ticket:
7,1,14

nearby tickets:
7,3,47
40,4,50
55,2,20
38,6,12'''

In [129]:
lines = get_input()

rules = lines[0]
my_ticket = [int(n) for n in lines[1].split('\n')[1].split(',')]
nearby_tickets = [[int(n) for n in tic.split(',')] for tic in lines[2].split('\n')[1:]]

In [130]:
df_nt = pd.DataFrame(nearby_tickets)

In [131]:
split_rules = re.findall('(.*): ([0-9]*)-([0-9]*) or ([0-9]*)-([0-9]*)', rules)

In [132]:
d_rules = {rule[0]:[int(n) for n in rule[1:]] for rule in split_rules}

In [133]:
fulfils = df_nt.isna()
for r in d_rules.values():
    fulfils |= (((df_nt >= r[0]) & (df_nt <= r[1])) | (((df_nt >= r[2]) & (df_nt <= r[3]))))

In [134]:
df_nt[~fulfils].sum().sum()

19093.0

## Part II

In [144]:
df_rt = df_nt[fulfils.sum(axis=1) == len(nt.columns)]
rule_cols = {}
while True:
    for rule,nums in d_rules.items():
        fulfils_rt = (((df_rt >= nums[0]) & (df_rt <= nums[1])) | (((df_rt >= nums[2]) & (df_rt <= nums[3]))))
        all_fulfilled = fulfils_rt.all() 
        if all_fulfilled.sum() == 1:
            col = all_fulfilled[all_fulfilled].index[0]
            rule_cols[rule] = col
            df_rt = df_rt.drop(columns=col)
    if df_rt.empty:
        break

In [149]:
dep_indices = [i for r,i in rule_cols.items() if 'departure' in r]

In [150]:
dep_indices

[19, 16, 10, 5, 11, 6]

In [154]:
print(reduce(lambda x,y: x*y, [my_ticket[ind] for ind in dep_indices]))

5311123569883
